In [2]:
import tkinter as tk
from tkinter import ttk

In [19]:
class ItemListBox(tk.Listbox):
    def __init__(self, *args, master, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self.__items = tk.StringVar(master, name='_items')
        self.config(listvariable=self.__items)

    @property
    def items(self):
        return self.getvar('_items')

    def get(self):
        return self._items.get()

    def add(self, item):
        if item not in self.items:
            self.insert('end', item)
    
    def add_many(self, *items):
        for item in items:
            self.add(item)

    def clear(self):
        self.delete(0, 'end')

    def remove(self, item):
        ix = self.items.index(item)
        self.delete(ix)

In [3]:
class OptionMenu(ttk.OptionMenu):
    __optnum = 0
    def __init__(self, *args, master, **kwargs):
        self.__optname = f'_options{self.__optnum}'
        self.__class__.__optnum += 1

        self._optionvar = tk.StringVar(master, name=self.__optname)
        super().__init__(*args, master=master, variable=self._optionvar, **kwargs)
        
        self._cache = []
    
    @property
    def items(self):
        return tuple(self._cache)
    
    def get(self):
        return self._optionvar.get()

    def add(self, item, default=False):
        if item not in self._cache:
            self._cache.append(item)
            self['menu'].add_command(
                label=item,
                command=tk._setit(self._optionvar, item),
            )
        if default:
            self._optionvar.set(item)
        else:
            self._optionvar.set(self._cache[0])
    
    def set_default(self, item):
        self._cache.index(item)
        if item in self._cache:
            self._optionvar.set(item)

    def remove(self, item):
        if item in self._cache:
            ix = item.index(item)
            self['menu'].delete(ix)
            self._cache.pop(ix)
            self.set_default(self.items[0])

    def add_many(self, *options):
        for opt in options:
            self.add(opt)

    @property
    def current_option(self):
        return self.getvar(self.__optname)

In [4]:
class ConfigBox(tk.Toplevel):
    def __init__(self, *args, master, **kwargs):
        super().__init__(*args, master=master, **kwargs)
        self.title("Configuration Panel")

        # design rest of config box
        panel = ttk.LabelFrame(self, text='Settings')
        panel.pack(padx=4, pady=4, side='top', anchor='n')
        # rest inside panel
        labels = ttk.Frame(panel)
        values = ttk.Frame(panel)
        labels.pack(side='left')
        values.pack(side='right')

        ttk.Label(labels, text='Extension:').grid(row=0, padx=4, pady=4)
        self._extbox = ext = ttk.Entry(values)
        ext.pack(padx=4, pady=2, fill='x')

        ttk.Label(labels, text='Backend:').grid(row=1, padx=4, pady=4)
        self._bops = bops = OptionMenu(master=values)
        bops.pack(padx=4, pady=2, fill='x')

        ttk.Label(labels, text='Key length:').grid(row=2, padx=4, pady=4)
        self._kops = kops = OptionMenu(master=values)
        kops.pack(padx=4, pady=2, fill='x')

        # apply, cancel
        btnframe = ttk.Frame(self)
        btnframe.pack(side='bottom', anchor='s', padx=4, pady=4)

        apply = ttk.Button(btnframe, text='Apply')
        apply.pack(side='left')
        apply.config(command=self.destroy)
        
        cancel = ttk.Button(btnframe, text='Cancel')
        apply.pack(side='left')
        apply.config(command=self.destroy)
        
        help = ttk.Button(btnframe, text='Help')
        apply.pack(side='left')
        apply.config(command=self.apply)        

    @property
    def backend_options(self):
        return self._bops

    @property
    def key_length_options(self):
        return self._kops
    
    @property
    def ext_entry_box(self):
        return self._extbox
    
    def apply(self):
        import re
        if not re.fullmatch(r"^\.[\w|\d]+", self._extbox.get()):
            #messagebox.showerror("Extension Error", "Invalid Extension")
            return
        vals = self._bops.get(), self._kops.get(), self._extbox.get()
        self.destroy()
        return vals

In [21]:
root = tk.Tk()


In [22]:
lb = ItemListBox(master=root)
lb.add_many('a', 'b', 'c')
lb.pack()


In [23]:
root.mainloop()

In [20]:

root.destroy()